# «SQL – Анализ базы данных сервиса для чтения книг по подписке»
**Описание проекта**<br>
Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.<br>
<br>
Компания решила быть на волне и купила крупный сервис для чтения книг по подписке.
<br>
Цели проекта: проанализировать базу данных. В базе данных информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Описание данных**

***Таблица `books`***

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

***Таблица `authors`***

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

***Таблица `publishers`***

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

***Таблица `ratings`***

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

***Таблица `reviews`***

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.


# План выполнения задания
1. Загрузить библиотеки, создать коннект к базе данных
1. Выполнить исследовательский анализ данных
   - Посчитать, сколько книг вышло после 1 января 2000 года;
   - Для каждой книги посчитайте количество обзоров и среднюю оценку;
   - Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
   - Определить автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками;
   - Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.
3. Общий вывод

## Импортируем библиотеки, посмотрим на данные

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine
from datetime import datetime

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

con=engine.connect()

In [3]:
# создадим функцию для выполнения запроса
def select(request):
    return pd.io.sql.read_sql(request, con = engine)

In [4]:
#Напишем простую функцию для просмотра информации
def information(data):
    display(select(data).head(5))
    select(data).info()
    print('Количество дубликатов:',select(data).duplicated().sum())
    print("Количество Пропусков в столбцах:\n", select(data).isna().sum())

In [5]:
books = '''
SELECT * FROM books
'''
select(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


In [6]:
authors = '''
SELECT * FROM authors
'''

In [7]:
publishers = '''
SELECT * FROM publishers
'''

In [8]:
ratings = '''
SELECT * FROM ratings
'''

In [9]:
reviews = '''
SELECT * FROM reviews
'''

In [10]:
information(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
Количество дубликатов: 0
Количество Пропусков в столбцах:
 book_id             0
author_id           0
title               0
num_pages           0
publication_date    0
publisher_id        0
dtype: int64


In [11]:
information(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB
Количество дубликатов: 0
Количество Пропусков в столбцах:
 author_id    0
author       0
dtype: int64


In [12]:
information(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB
Количество дубликатов: 0
Количество Пропусков в столбцах:
 publisher_id    0
publisher       0
dtype: int64


In [13]:
information(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB
Количество дубликатов: 0
Количество Пропусков в столбцах:
 rating_id    0
book_id      0
username     0
rating       0
dtype: int64


In [14]:
information(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB
Количество дубликатов: 0
Количество Пропусков в столбцах:
 review_id    0
book_id      0
username     0
text         0
dtype: int64


**Промежуточный вывод:** <br>
На данные мы посмотрели, пропусков и дубликатов нигде не обнаружили. Можно сказать, что таблицы соответствуют заявленной схеме.

## Исследовательский анализ данных
### Посчитаем сколько книг вышло после 1 января 2000 года

In [15]:
print_books = '''
SELECT 
    COUNT(*) AS cnt_books
FROM 
    books
WHERE 
    publication_date > '2000-01-01';
'''
select(print_books)

,cnt_books
0,819


Согласно нашим данным, после 1 января 2000 года вышло 819 книжных экземпляров

### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [16]:
ratings = '''
SELECT 
  books.book_id, 
  title, 
  COUNT(DISTINCT review_id) AS count, 
  ROUND(
    AVG(rating), 
    2
  ) AS avg 
FROM 
  books 
  LEFT JOIN ratings ON books.book_id = ratings.book_id 
  LEFT JOIN reviews ON books.book_id = reviews.book_id 
GROUP BY 
  books.book_id, 
  title 
ORDER BY 
  count DESC
'''

select(ratings)

,book_id,title,count,avg
0,948,Twilight (Twilight #1),7,3.66
1,963,Water for Elephants,6,3.98
2,734,The Glass Castle,6,4.21
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,695,The Curious Incident of the Dog in the Night-Time,6,4.08
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
996,808,The Natural Way to Draw,0,3.00
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
998,221,Essential Tales and Poems,0,4.00


Самая обозреваемая книга Twilight (Twilight #1) Самая плохая оценка (1.5) у книги Harvesting the Heart. Можно заметить у некоторых книг, что люди проставили оценку, но не написали обзор. Это плохо, можно специально занижать рейтинг.

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц
Таким образом мы исключим из анализа брошюры

In [17]:
book_publisher = '''
SELECT 
  publishers.publisher AS publ, 
  COUNT(books.book_id) AS count 
FROM 
  publishers FULL 
  OUTER JOIN books ON books.publisher_id = publishers.publisher_id 
WHERE 
  books.num_pages > 50 
GROUP BY 
  publ 
ORDER BY 
  count DESC
'''
select(book_publisher)

,publ,count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Turtleback,1
330,Atheneum Books for Young Readers: Richard Jack...,1
331,Penguin Signet,1
332,Victor Gollancz,1


In [18]:
book_publisher = '''
SELECT
  publishers.publisher AS publ, 
  COUNT(books.book_id) AS count 
FROM 
  publishers FULL 
  OUTER JOIN books ON books.publisher_id = publishers.publisher_id 
WHERE 
  books.num_pages > 50 
GROUP BY 
  publ 
ORDER BY 
  count DESC 
LIMIT 
  1
'''
select(book_publisher)

,publ,count
0,Penguin Books,42


Penguin Books оказалось самым работоспособным издательством, которое выпустило 42 книги.<br>
<br>

Дополнительно можем выделить ТОП-5 издательств по выпущенным книгам: 
1. Penguin Books - 42 книги
1. Vintage -	31 книга
1. Grand Central Publishing -	25 книг
1. Penguin Classics	- 24 книги
1. Ballantine Books -	19 книг

### Определим автора с самой высокой средней оценкой книг 

Будем учитывать только книги с 50 и более оценками

In [19]:
author_rating_var = '''
WITH i AS (
  SELECT 
    authors.author, 
    books.book_id, 
    AVG(ratings.rating) AS avg_rating 
  FROM 
    authors FULL 
    JOIN books ON books.author_id = authors.author_id FULL 
    JOIN ratings ON ratings.book_id = books.book_id 
  GROUP BY 
    author, 
    books.book_id 
  HAVING 
    COUNT(ratings.rating) >= 50
) 
SELECT 
  i.author, 
  ROUND(
    AVG(i.avg_rating), 
    4
  ) AS avg_rating 
FROM 
  i 
GROUP BY 
  author 
ORDER BY 
  avg_rating DESC 
LIMIT 
  1;
'''
select(author_rating_var)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.2838


Джоан Роулинг известная по книгам Гарри Поттера возгавляет ТОП с рейтингом в 4.28.

In [20]:
#По тз нужно добавить 2 варианта с HAVING и без
author_rating_var2 = '''
WITH i AS (
  WITH temp AS (
    SELECT 
      b.book_id, 
      b.author_id, 
      aut.author, 
      rat.rating, 
      COUNT (rating) over (partition BY b.book_id) AS count 
    FROM 
      books AS b 
      RIGHT join ratings AS rat on rat.book_id = b.book_id 
      RIGHT join author AS aut on aut.author_id = b.author_id
  ) 
  SELECT 
    * 
  FROM 
    temp 
  WHERE 
    count > 50
) 
SELECT 
  author, 
  ROUND(
    AVG(rating), 
    4
  ) 
FROM 
  i 
GROUP BY 
  author 
ORDER BY 
  avg(rating) DESC 
LIMIT 
  1;
'''
select(author_rating_var2)

,author,round
0,J.K. Rowling/Mary GrandPré,4.2871


В принципе лидером осталась Джоан Роулинг - J.K. Rowling/Mary GrandPré с рейтингом 4.2871

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [21]:
avg_rev = '''
SELECT 
  ROUND(
    AVG(a.count), 
    2
  ) 
FROM 
  (
    SELECT 
      username, 
      COUNT(review_id) AS count 
    FROM 
      reviews 
    WHERE 
      username IN (
        SELECT 
          username 
        FROM 
          ratings 
        GROUP BY 
          username 
        HAVING 
          COUNT(rating_id) > 48
      ) 
    GROUP BY 
      username
  ) AS a;
'''
select(avg_rev)

,round
0,24.0


Итого, пользователи,которые ставят 48 оценок, на половину из них пишут рецензию. А именно на 24 книги

## Итоговый вывод:
- Пропусков и дубликатов в таблицах найдено не было. 
- Можно сказать, что таблицы соответствуют заявленной схеме
- Согласно нашим данным, после 1 января 2000 года вышло 819 книжных экземпляров
- Penguin Books оказалось самым работоспособным издательством, которое выпустило 42 книги.
- Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок, составило 24.
- Автор с самой высокой средней оценкой книг - Джоан Роулинг(J.K. Rowling/Mary GrandPré) с оценкой 4.28<br><br><br>

*Дополнительно:*
- Самая обозреваемая книга Twilight (Twilight #1) Самая плохая оценка (1.5) у книги Harvesting the Heart. Можно заметить у некоторых книг, что люди проставили оценку, но не написали обзор. Это плохо, можно таким образом специально занижать рейтинг.
- Можем выделить ТОП-5 издательств по выпущенным книгам: <br>
    1. Penguin Books - 42 книги
    1. Vintage -	31 книга
    1. Grand Central Publishing -	25 книг
    1. Penguin Classics	- 24 книги
    1. Ballantine Books -	19 книг